### ACTUAL

In [1]:
import os
import json

def delete_model_jsons(model_name):
    """
    Deletes JSON files for the specified model name across all indicators and countries.
    Replaces spaces in country names with underscores.

    Args:
        model_name (str): The name of the model (e.g., "XGBoost", "Prophet").

    Returns:
        None
    """
    # Load country names and indicators
    with open("../countries.json", "r") as f:
        country_names = json.load(f)
    with open("../indicators.json", "r") as f:
        indicators = json.load(f)

    # Define the base path for the parameter files
    base_dir = "../best_params"

    # Iterate over all indicators and countries to delete JSON files
    for indicator in indicators.keys():
        for country in country_names.keys():
            # Replace spaces in the country name with underscores

            # Construct the file path
            json_file_path = os.path.join(base_dir, indicator, f"{model_name}_{country}.json")
            
            # Check if the file exists and delete it
            if os.path.exists(json_file_path):
                try:
                    os.remove(json_file_path)
                    print(f"Deleted: {json_file_path}")
                except Exception as e:
                    print(f"Error deleting {json_file_path}: {e}")
            else:
                pass

# Example usage:
delete_model_jsons("XGBoost")


Deleted: ../best_params\GDP per Capita (USD)\XGBoost_Czech Republic.json
Deleted: ../best_params\GDP per Capita (USD)\XGBoost_Hungary.json
Deleted: ../best_params\GDP per Capita (USD)\XGBoost_Poland.json
Deleted: ../best_params\GDP per Capita (USD)\XGBoost_Slovakia.json
Deleted: ../best_params\GDP per Capita (USD)\XGBoost_Germany.json
Deleted: ../best_params\GDP per Capita (USD)\XGBoost_Austria.json
Deleted: ../best_params\GDP per Capita (USD)\XGBoost_France.json
Deleted: ../best_params\GDP per Capita (USD)\XGBoost_Italy.json
Deleted: ../best_params\GDP (USD)\XGBoost_Czech Republic.json
Deleted: ../best_params\GDP (USD)\XGBoost_Hungary.json
Deleted: ../best_params\GDP (USD)\XGBoost_Poland.json
Deleted: ../best_params\GDP (USD)\XGBoost_Slovakia.json
Deleted: ../best_params\GDP (USD)\XGBoost_Germany.json
Deleted: ../best_params\GDP (USD)\XGBoost_Austria.json
Deleted: ../best_params\GDP (USD)\XGBoost_France.json
Deleted: ../best_params\GDP (USD)\XGBoost_Italy.json
Deleted: ../best_params\

In [3]:
import os
import json
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFE
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

def save_plot(train, test_y, predictions, country, indicator, model_name):
    """Function to save the plot in both Indicators and Countries folders."""

    model_colors = {
    "ARIMA": "blue",
    "Holt_Winters": "yellow",
    "LSTM": "black",
    "XGBoost": "pink",
    "Prophet": "brown"
}

    # Plotting predicted vs actual
    plt.figure(figsize=(10, 6))
    if model_name == "Prophet":
        plt.plot(train['ds'], train['y'], label='Train Data', color='green', linestyle='--')
        plt.plot(test_y['ds'], test_y['y'], label='Actual', color='red', linestyle='--')
        plt.plot(test_y['ds'], predictions, label=f'Predicted({model_name})', color=f'{model_colors["Prophet"]}', 
                 linestyle='-', marker='o')
    else:
        plt.plot(train.index, train, label='Train Data', color='green', linestyle='--')
        plt.plot(test_y.index, test_y, label='Actual', color='red', linestyle='--')
        plt.plot(test_y.index, predictions, label=f'Predicted({model_name})', color=f'{model_colors[model_name]}', 
                 linestyle='-', marker='o')
    

    
    plt.title(f'Predicted({model_name}) vs Actual for {country} - {indicator}')
    plt.xlabel('Year')
    plt.ylabel('Value')
    plt.legend()

    # Create subfolder for the indicator if it doesn't exist
    indicator_folder = os.path.join('../images', 'model_plot', 'Indicators', indicator)
    os.makedirs(indicator_folder, exist_ok=True)
    
    # Save the plot in the Indicators folder with dynamic model name
    plot_filename_indicator = os.path.join(indicator_folder, f'{model_name}_{country.replace(" ", "_")}_{indicator.replace(" ", "_")}.png')
    plt.savefig(plot_filename_indicator)

    # Create subfolder for the country if it doesn't exist
    country_folder = os.path.join('../images', 'model_plot', 'Countries', country)
    os.makedirs(country_folder, exist_ok=True)
    
    # Save the same plot in the Countries folder with dynamic model name
    plot_filename_country = os.path.join(country_folder, f'{model_name}_{country.replace(" ", "_")}_{indicator.replace(" ", "_")}.png')
    plt.savefig(plot_filename_country)

    plt.close()

def train_xgboost(train_x, train_y, test_x, test_y, country, indicator):
    params_dir = os.path.join("../best_params", indicator)
    params_file = os.path.join(params_dir, f"XGBoost_{country}.json")
    best_params = None

    # Check if the parameter file exists
    if os.path.exists(params_file):
        print(f"Loading parameters from {params_file}...")
        with open(params_file, "r") as f:
            best_params = json.load(f)
    else:
        print("No saved parameters found, performing grid search...")

        # Define the parameter grid for GridSearchCV
        param_grid = {
            'n_estimators': [50, 100, 600, 700, 800, 900, 1000],
            'learning_rate': [0.01, 0.05],
            'max_depth': [3, 4, 5, 6]
        }

        # Initialize the XGBoost regressor
        model = xgb.XGBRegressor(objective='reg:squarederror')

        # Perform GridSearchCV to find the best hyperparameters
        grid_search = GridSearchCV(model, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=0)
        grid_search.fit(train_x, train_y)

        # Get the best parameters and save them to JSON
        best_params = grid_search.best_params_
        os.makedirs(params_dir, exist_ok=True)
        with open(params_file, "w") as f:
            json.dump(best_params, f, indent=4)

    # Train the best model using the loaded or selected parameters
    best_model = xgb.XGBRegressor(
        n_estimators=best_params['n_estimators'],
        learning_rate=best_params['learning_rate'],
        max_depth=best_params['max_depth'],
        objective='reg:squarederror'
    )

    # Perform Recursive Feature Elimination (RFE)
    top_n_features = 4
    selector = RFE(estimator=best_model, n_features_to_select=top_n_features)
    selector = selector.fit(train_x, train_y)

    # Get selected feature names
    selected_features = train_x.columns[selector.support_]

    # Save selected features to JSON
    features_file = os.path.join(params_dir, f"SelectedFeatures_{country}.json")
    with open(features_file, "w") as f:
        json.dump(selected_features.tolist(), f, indent=4)

    # Transform data based on selected features
    selected_train_x = selector.transform(train_x)
    selected_test_x = selector.transform(test_x)

    # Train the model again with the selected features
    best_model.fit(selected_train_x, train_y)

    # Make predictions on the test set using the selected features
    predictions = best_model.predict(selected_test_x)

    # Save the plot
    save_plot(train_y, test_y, predictions, country, indicator, model_name="XGBoost")

    # Calculate the RMSE on the test set
    return np.sqrt(mean_squared_error(test_y, predictions)), predictions




with open("../countries.json", "r") as f:
    country_names = json.load(f)

with open("../indicators.json", "r") as f:
    indicators = json.load(f)

data_folder = "../data/base"
model_errors_rmse = {}
log_data = []
country_indicators_plots = {}
for country, country_code in country_names.items():
    for indicator, indicator_code in indicators.items():
        filename = f"{country.replace(' ', '_')}_{indicator.replace(' ', '_')}.parquet"
        filepath = os.path.join(data_folder, filename)
        
        if os.path.exists(filepath):
            df = pd.read_parquet(filepath)
            if 'Year' in df.columns and 'Value' in df.columns:
                df = df.set_index('Year').sort_index()
                df.index = pd.to_datetime(df.index, format='%Y')
                df = df.dropna()
                df = df.drop('Indicator', axis = 1)
                df_original = df.copy()
                
                for lag in range(1, 6):  
                    df[f'lag_{lag}'] = df['Value'].shift(lag)
                
                df['expanding_mean'] = df['Value'].expanding().mean()
                df['expanding_std'] = df['Value'].expanding().std()
                df['expanding_max'] = df['Value'].expanding().max()
                df['expanding_min'] = df['Value'].expanding().min()
                
                window_size = 3  
                df['rolling_mean'] = df['Value'].rolling(window=window_size, min_periods=1).mean()
                df['rolling_std'] = df['Value'].rolling(window=window_size, min_periods=1).std()
                df['rolling_max'] = df['Value'].rolling(window=window_size, min_periods=1).max()
                df['rolling_min'] = df['Value'].rolling(window=window_size, min_periods=1).min()
                
                
                #df = df.dropna()
                train_size = int(len(df) * 0.8)
                
                train_xgb, test_xgb = df.iloc[:train_size], df.iloc[train_size:]
                feature_columns = ['rolling_mean', 'rolling_std', 'rolling_max', 'rolling_min', 
                                   'expanding_mean', 'expanding_std', 'expanding_max', 'expanding_min',
                                   'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5']
                train_x, train_y = train_xgb[feature_columns], train_xgb['Value']
                test_x, test_y = test_xgb[feature_columns], test_xgb['Value']
                
                model_errors_rmse[(country, indicator)] = {}
                model_errors_rmse[(country, indicator)]['XGBoost'], xgb_pred = train_xgboost(train_x, train_y, test_x, test_y , country,indicator)
                
                
                sorted_models = sorted(model_errors_rmse[(country, indicator)].items(), key=lambda x: x[1])
                log_current_data = []
                for rank, (model_name, rmse) in enumerate(sorted_models, start=1):
                    log_data.append([country, indicator, model_name, rmse, rank])
                    log_current_data.append([country, indicator, model_name, rmse, rank])


from datetime import datetime
model ="XGBoost"
log_dir = f"../data/{model}_train"
os.makedirs(log_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d--%H-%M")
log_filename = os.path.join(log_dir, f"{model}_error_log_{timestamp}.csv")

log_df = pd.DataFrame(log_data, columns=['Country', 'Indicator', 'Model', 'RMSE', 'Rank'])
log_df.to_csv(log_filename, index=False)


No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid sea

### TEST

In [8]:
import os
import json
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFE
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

def save_plot(train, test_y, predictions, country, indicator, model_name):
    """Function to save the plot in both Indicators and Countries folders."""

    model_colors = {
    "ARIMA": "blue",
    "Holt_Winters": "yellow",
    "LSTM": "black",
    "XGBoost": "pink",
    "Prophet": "brown"
}

    # Plotting predicted vs actual
    plt.figure(figsize=(10, 6))
    if model_name == "Prophet":
        plt.plot(train['ds'], train['y'], label='Train Data', color='green', linestyle='--')
        plt.plot(test_y['ds'], test_y['y'], label='Actual', color='red', linestyle='--')
        plt.plot(test_y['ds'], predictions, label=f'Predicted({model_name})', color=f'{model_colors["Prophet"]}', 
                 linestyle='-', marker='o')
    else:
        plt.plot(train.index, train, label='Train Data', color='green', linestyle='--')
        plt.plot(test_y.index, test_y, label='Actual', color='red', linestyle='--')
        plt.plot(test_y.index, predictions, label=f'Predicted({model_name})', color=f'{model_colors[model_name]}', 
                 linestyle='-', marker='o')
    

    
    plt.title(f'Predicted({model_name}) vs Actual for {country} - {indicator}')
    plt.xlabel('Year')
    plt.ylabel('Value')
    plt.legend()

    # Create subfolder for the indicator if it doesn't exist
    indicator_folder = os.path.join('../images', 'model_plot', 'Indicators', indicator)
    os.makedirs(indicator_folder, exist_ok=True)
    
    # Save the plot in the Indicators folder with dynamic model name
    plot_filename_indicator = os.path.join(indicator_folder, f'{model_name}_{country.replace(" ", "_")}_{indicator.replace(" ", "_")}.png')
    plt.savefig(plot_filename_indicator)

    # Create subfolder for the country if it doesn't exist
    country_folder = os.path.join('../images', 'model_plot', 'Countries', country)
    os.makedirs(country_folder, exist_ok=True)
    
    # Save the same plot in the Countries folder with dynamic model name
    plot_filename_country = os.path.join(country_folder, f'{model_name}_{country.replace(" ", "_")}_{indicator.replace(" ", "_")}.png')
    plt.savefig(plot_filename_country)

    plt.close()

def train_xgboost(train_x, train_y, test_x, test_y, country, indicator):
    params_dir = os.path.join("../best_params", indicator)
    params_file = os.path.join(params_dir, f"XGBoost_{country}.json")
    best_params = None

    # Check if the parameter file exists
    if os.path.exists(params_file):
        print(f"Loading parameters from {params_file}...")
        with open(params_file, "r") as f:
            best_params = json.load(f)
    else:
        print("No saved parameters found, performing grid search...")

        # Define parameter grid for GridSearchCV
        param_grid = {
            'n_estimators': [50, 100, 200, 500, 700, 900],
            'learning_rate': [0.01, 0.05, 0.1],
            'max_depth': [3, 5, 7],
            'min_child_weight': [1, 3],
            'gamma': [0, 0.1, 0.3],
            'subsample': [0.5,0.7,0.9,1],
            'colsample_bytree': [0.8, 1.0]
        }

        # Initialize the model
        model = xgb.XGBRegressor(
            objective='reg:squarederror', 
            random_state=42
        )

        # Set up the grid search with cross-validation
        grid_search = GridSearchCV(
            estimator=model,
            param_grid=param_grid,
            cv=3,
            scoring='neg_root_mean_squared_error',
            verbose=0,
            n_jobs=-1
        )

        # Fit grid search
        grid_search.fit(train_x, train_y)

        # Get the best parameters from grid search
        best_params = grid_search.best_params_

        # Save the best parameters to JSON if found
        if best_params:
            os.makedirs(params_dir, exist_ok=True)
            with open(params_file, "w") as f:
                json.dump(best_params, f, indent=4)

    # Train the best model using loaded or selected parameters
    if best_params:
        best_model = xgb.XGBRegressor(
            n_estimators=best_params['n_estimators'],
            learning_rate=best_params['learning_rate'],
            max_depth=best_params['max_depth'],
            min_child_weight=best_params['min_child_weight'],
            gamma=best_params['gamma'],
            subsample=best_params['subsample'],
            colsample_bytree=best_params['colsample_bytree'],
            objective='reg:squarederror',
            random_state=42
        )

        # Perform Recursive Feature Elimination (RFE)
        rfe = RFE(estimator=best_model, n_features_to_select=3)
        rfe.fit(train_x, train_y)

        # Get the selected features
        selected_features = train_x.columns[rfe.support_]


        # Train the model on the selected features
        best_model.fit(train_x[selected_features], train_y, verbose=0)

        # Make predictions
        #predictions = best_model.predict(test_x[selected_features])

        predictions_rfe = best_model.predict(test_x[selected_features])

        # Calculate RMSE for the RFE model
        rmse_rfe = np.sqrt(mean_squared_error(test_y, predictions_rfe))

        # Feature importance analysis
        feature_importances = best_model.feature_importances_
        important_features = selected_features[feature_importances > 0.1]

        if len(important_features) > 0:
            # Retrain the model using only features with importance > 0.1
            best_model.fit(train_x[important_features], train_y, verbose=0)
            predictions_important = best_model.predict(test_x[important_features])

            # Calculate RMSE for the model trained on important features
            rmse_important = np.sqrt(mean_squared_error(test_y, predictions_important))

        if rmse_important >rmse_rfe:
            predictions = predictions_important
        else:
            predictions = predictions_rfe
        # Save the plot
        save_plot(train_y, test_y, predictions, country, indicator, model_name="XGBoost")

        return np.sqrt(mean_squared_error(test_y, predictions)), predictions
    else:
        print("No suitable parameters found.")
        return None, None







with open("../countries.json", "r") as f:
    country_names = json.load(f)

with open("../indicators.json", "r") as f:
    indicators = json.load(f)

data_folder = "../data/base"
model_errors_rmse = {}
log_data = []
country_indicators_plots = {}
for country, country_code in country_names.items():
    for indicator, indicator_code in indicators.items():
        filename = f"{country.replace(' ', '_')}_{indicator.replace(' ', '_')}.parquet"
        filepath = os.path.join(data_folder, filename)
        
        if os.path.exists(filepath):
            df = pd.read_parquet(filepath)
            if 'Year' in df.columns and 'Value' in df.columns:
                df = df.set_index('Year').sort_index()
                df.index = pd.to_datetime(df.index, format='%Y')
                df = df.dropna()
                df = df.drop('Indicator', axis = 1)
                df_original = df.copy()
                
                for lag in range(1, 4):  
                    df[f'lag_{lag}'] = df['Value'].shift(lag)
                
                df['expanding_mean'] = df['Value'].expanding().mean()
                df['expanding_std'] = df['Value'].expanding().std()
                df['expanding_max'] = df['Value'].expanding().max()
                df['expanding_min'] = df['Value'].expanding().min()
                
                window_size = 3  
                df['rolling_mean'] = df['Value'].rolling(window=window_size, min_periods=1).mean()
                df['rolling_std'] = df['Value'].rolling(window=window_size, min_periods=1).std()
                df['rolling_max'] = df['Value'].rolling(window=window_size, min_periods=1).max()
                df['rolling_min'] = df['Value'].rolling(window=window_size, min_periods=1).min()
                
                
                #df = df.dropna()
                train_size = int(len(df) * 0.8)
                
                train_xgb, test_xgb = df.iloc[:train_size], df.iloc[train_size:]
                feature_columns = ['rolling_mean','rolling_max', 'rolling_min','rolling_std' ,
                                   'expanding_mean', 'expanding_max', 'expanding_min','expanding_std',
                                   'lag_1', 'lag_2', 'lag_3', ]
                train_x, train_y = train_xgb[feature_columns], train_xgb['Value']
                test_x, test_y = test_xgb[feature_columns], test_xgb['Value']
                
                model_errors_rmse[(country, indicator)] = {}
                model_errors_rmse[(country, indicator)]['XGBoost'], xgb_pred = train_xgboost(train_x, train_y, test_x, test_y , country,indicator)
                
                
                sorted_models = sorted(model_errors_rmse[(country, indicator)].items(), key=lambda x: x[1])
                log_current_data = []
                for rank, (model_name, rmse) in enumerate(sorted_models, start=1):
                    log_data.append([country, indicator, model_name, rmse, rank])
                    log_current_data.append([country, indicator, model_name, rmse, rank])


from datetime import datetime
model ="XGBoost"
log_dir = f"../data/{model}_train"
os.makedirs(log_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d--%H-%M")
log_filename = os.path.join(log_dir, f"{model}_error_log_{timestamp}.csv")

log_df = pd.DataFrame(log_data, columns=['Country', 'Indicator', 'Model', 'RMSE', 'Rank'])
log_df.to_csv(log_filename, index=False)


No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid search...
No saved parameters found, performing grid sea

### STARSIE VERZIE

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFE
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

def save_plot(train, test_y, predictions, country, indicator, model_name):
    """Function to save the plot in both Indicators and Countries folders."""

    model_colors = {
    "ARIMA": "blue",
    "Holt_Winters": "yellow",
    "LSTM": "black",
    "XGBoost": "pink",
    "Prophet": "brown"
}

    # Plotting predicted vs actual
    plt.figure(figsize=(10, 6))
    if model_name == "Prophet":
        plt.plot(train['ds'], train['y'], label='Train Data', color='green', linestyle='--')
        plt.plot(test_y['ds'], test_y['y'], label='Actual', color='red', linestyle='--')
        plt.plot(test_y['ds'], predictions, label=f'Predicted({model_name})', color=f'{model_colors["Prophet"]}', 
                 linestyle='-', marker='o')
    else:
        plt.plot(train.index, train, label='Train Data', color='green', linestyle='--')
        plt.plot(test_y.index, test_y, label='Actual', color='red', linestyle='--')
        plt.plot(test_y.index, predictions, label=f'Predicted({model_name})', color=f'{model_colors[model_name]}', 
                 linestyle='-', marker='o')
    

    
    plt.title(f'Predicted({model_name}) vs Actual for {country} - {indicator}')
    plt.xlabel('Year')
    plt.ylabel('Value')
    plt.legend()

    # Create subfolder for the indicator if it doesn't exist
    indicator_folder = os.path.join('../images', 'model_plot', 'Indicators', indicator)
    os.makedirs(indicator_folder, exist_ok=True)
    
    # Save the plot in the Indicators folder with dynamic model name
    plot_filename_indicator = os.path.join(indicator_folder, f'{model_name}_{country.replace(" ", "_")}_{indicator.replace(" ", "_")}.png')
    plt.savefig(plot_filename_indicator)

    # Create subfolder for the country if it doesn't exist
    country_folder = os.path.join('../images', 'model_plot', 'Countries', country)
    os.makedirs(country_folder, exist_ok=True)
    
    # Save the same plot in the Countries folder with dynamic model name
    plot_filename_country = os.path.join(country_folder, f'{model_name}_{country.replace(" ", "_")}_{indicator.replace(" ", "_")}.png')
    plt.savefig(plot_filename_country)

    plt.close()

def train_xgboost(train_x, train_y, test_x, test_y, country, indicator):
    params_dir = os.path.join("../best_params", indicator)
    params_file = os.path.join(params_dir, f"XGBoost_{country}.json")
    best_params = None

    # Check if the parameter file exists
    if os.path.exists(params_file):
        print(f"Loading parameters from {params_file}...")
        with open(params_file, "r") as f:
            best_params = json.load(f)
    else:
        print("No saved parameters found, performing grid search...")

        # Define parameter grid for GridSearchCV
        param_grid = {
            'n_estimators': [50, 100, 200, 400, 500, 700, 900],
            'learning_rate': [0.01, 0.05, 0.1],
            'max_depth': [3, 5, 7],
            'min_child_weight': [1, 3],
            'gamma': [0, 0.1, 0.3],
            'subsample': [0.8, 0.9],
            'colsample_bytree': [0.8, 1.0]
        }

        # Split train data into training and validation sets
        train_x_split, val_x_split, train_y_split, val_y_split = train_test_split(
            train_x, train_y, test_size=0.2, random_state=42
        )

        # Initialize the model
        model = xgb.XGBRegressor(
            objective='reg:squarederror', 
            random_state=42,
            eval_metric='rmse',
            early_stopping_rounds=30
        )

        # Set up the grid search with cross-validation
        grid_search = GridSearchCV(
            estimator=model,
            param_grid=param_grid,
            cv=3,
            scoring='neg_root_mean_squared_error',
            verbose=0,
            n_jobs=-1
        )

        # Fit grid search
        grid_search.fit(
            train_x_split, 
            train_y_split,
            eval_set=[(val_x_split, val_y_split)],
            verbose=0
        )

        # Get the best parameters from grid search
        best_params = grid_search.best_params_

        # Save the best parameters to JSON if found
        if best_params:
            os.makedirs(params_dir, exist_ok=True)
            with open(params_file, "w") as f:
                json.dump(best_params, f, indent=4)

    # Train the best model using loaded or selected parameters
    if best_params:
        best_model = xgb.XGBRegressor(
            n_estimators=best_params['n_estimators'],
            learning_rate=best_params['learning_rate'],
            max_depth=best_params['max_depth'],
            min_child_weight=best_params['min_child_weight'],
            gamma=best_params['gamma'],
            subsample=best_params['subsample'],
            colsample_bytree=best_params['colsample_bytree'],
            objective='reg:squarederror',
            random_state=42,
            eval_metric='rmse'
        )

        rfe = RFE(estimator=best_model, n_features_to_select=3)
        rfe.fit(train_x, train_y)

        # Get the selected features
        selected_features = train_x.columns[rfe.support_]

        print("Top 3 Selected Features:")
        for feature in selected_features:
            print(feature)

        # Train the model on the selected features
        best_model.fit(train_x[selected_features], train_y, verbose=0)

        # Make predictions
        predictions = best_model.predict(test_x[selected_features])

        # Save the plot
        save_plot(train_y, test_y, predictions, country, indicator, model_name="XGBoost")

        return np.sqrt(mean_squared_error(test_y, predictions)), predictions
        
    else:
        print("No suitable parameters found.")
        return None, None






with open("../countries.json", "r") as f:
    country_names = json.load(f)

with open("../indicators.json", "r") as f:
    indicators = json.load(f)

data_folder = "../data/base"
model_errors_rmse = {}
log_data = []
country_indicators_plots = {}
for country, country_code in country_names.items():
    for indicator, indicator_code in indicators.items():
        filename = f"{country.replace(' ', '_')}_{indicator.replace(' ', '_')}.parquet"
        filepath = os.path.join(data_folder, filename)
        
        if os.path.exists(filepath):
            df = pd.read_parquet(filepath)
            if 'Year' in df.columns and 'Value' in df.columns:
                df = df.set_index('Year').sort_index()
                df.index = pd.to_datetime(df.index, format='%Y')
                df = df.dropna()
                df = df.drop('Indicator', axis = 1)
                df_original = df.copy()
                
                for lag in range(1, 4):  
                    df[f'lag_{lag}'] = df['Value'].shift(lag)
                
                df['expanding_mean'] = df['Value'].expanding().mean()
                df['expanding_std'] = df['Value'].expanding().std()
                df['expanding_max'] = df['Value'].expanding().max()
                df['expanding_min'] = df['Value'].expanding().min()
                
                window_size = 3  
                df['rolling_mean'] = df['Value'].rolling(window=window_size, min_periods=1).mean()
                df['rolling_std'] = df['Value'].rolling(window=window_size, min_periods=1).std()
                df['rolling_max'] = df['Value'].rolling(window=window_size, min_periods=1).max()
                df['rolling_min'] = df['Value'].rolling(window=window_size, min_periods=1).min()
                
                
                df = df.dropna()
                train_size = int(len(df) * 0.8)
                
                train_xgb, test_xgb = df.iloc[:train_size], df.iloc[train_size:]
                feature_columns = ['rolling_mean','rolling_max', 'rolling_min','rolling_std' ,
                                   'expanding_mean', 'expanding_max', 'expanding_min','expanding_std',
                                   'lag_1', 'lag_2', 'lag_3', ]
                train_x, train_y = train_xgb[feature_columns], train_xgb['Value']
                test_x, test_y = test_xgb[feature_columns], test_xgb['Value']
                
                model_errors_rmse[(country, indicator)] = {}
                model_errors_rmse[(country, indicator)]['XGBoost'], xgb_pred = train_xgboost(train_x, train_y, test_x, test_y , country,indicator)
                
                
                sorted_models = sorted(model_errors_rmse[(country, indicator)].items(), key=lambda x: x[1])
                log_current_data = []
                for rank, (model_name, rmse) in enumerate(sorted_models, start=1):
                    log_data.append([country, indicator, model_name, rmse, rank])
                    log_current_data.append([country, indicator, model_name, rmse, rank])


from datetime import datetime
model ="XGBoost"
log_dir = f"../data/{model}_train"
os.makedirs(log_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d--%H-%M")
log_filename = os.path.join(log_dir, f"{model}_error_log_{timestamp}.csv")

log_df = pd.DataFrame(log_data, columns=['Country', 'Indicator', 'Model', 'RMSE', 'Rank'])
log_df.to_csv(log_filename, index=False)


In [ ]:
def train_xgboost(train_x, train_y, test_x, test_y, country, indicator):
    params_dir = os.path.join("../best_params", indicator)
    params_file = os.path.join(params_dir, f"XGBoost_{country}.json")
    best_params = None

    # Check if the parameter file exists
    if os.path.exists(params_file):
        print(f"Loading parameters from {params_file}...")
        with open(params_file, "r") as f:
            best_params = json.load(f)
    else:
        print("No saved parameters found, performing grid search...")

        # Define parameter grid for GridSearchCV
        param_grid = {
            'n_estimators': [50, 100, 200, 400, 500, 700, 900],
            'learning_rate': [0.01, 0.05, 0.1],
            'max_depth': [3, 5, 7],
            'min_child_weight': [1, 3],
            'gamma': [0, 0.1, 0.3],
            'subsample': [0.8, 0.9],
            'colsample_bytree': [0.8, 1.0]
        }

        # Initialize the model
        model = xgb.XGBRegressor(
            objective='reg:squarederror', 
            random_state=42
        )

        # Set up the grid search with cross-validation
        grid_search = GridSearchCV(
            estimator=model,
            param_grid=param_grid,
            cv=3,
            scoring='neg_root_mean_squared_error',
            verbose=0,
            n_jobs=-1
        )

        # Fit grid search
        grid_search.fit(train_x, train_y)

        # Get the best parameters from grid search
        best_params = grid_search.best_params_

        # Save the best parameters to JSON if found
        if best_params:
            os.makedirs(params_dir, exist_ok=True)
            with open(params_file, "w") as f:
                json.dump(best_params, f, indent=4)

    # Train the best model using loaded or selected parameters
    if best_params:
        best_model = xgb.XGBRegressor(
            n_estimators=best_params['n_estimators'],
            learning_rate=best_params['learning_rate'],
            max_depth=best_params['max_depth'],
            min_child_weight=best_params['min_child_weight'],
            gamma=best_params['gamma'],
            subsample=best_params['subsample'],
            colsample_bytree=best_params['colsample_bytree'],
            objective='reg:squarederror',
            random_state=42
        )

        # Perform Recursive Feature Elimination (RFE)
        rfe = RFE(estimator=best_model, n_features_to_select=3)
        rfe.fit(train_x, train_y)

        # Get the selected features
        selected_features = train_x.columns[rfe.support_]


        # Train the model on the selected features
        best_model.fit(train_x[selected_features], train_y, verbose=0)

        # Make predictions
        predictions = best_model.predict(test_x[selected_features])

        # Save the plot
        save_plot(train_y, test_y, predictions, country, indicator, model_name="XGBoost")

        return np.sqrt(mean_squared_error(test_y, predictions)), predictions
    else:
        print("No suitable parameters found.")
        return None, None







with open("../countries.json", "r") as f:
    country_names = json.load(f)

with open("../indicators.json", "r") as f:
    indicators = json.load(f)

data_folder = "../data/base"
model_errors_rmse = {}
log_data = []
country_indicators_plots = {}
for country, country_code in country_names.items():
    for indicator, indicator_code in indicators.items():
        filename = f"{country.replace(' ', '_')}_{indicator.replace(' ', '_')}.parquet"
        filepath = os.path.join(data_folder, filename)
        
        if os.path.exists(filepath):
            df = pd.read_parquet(filepath)
            if 'Year' in df.columns and 'Value' in df.columns:
                df = df.set_index('Year').sort_index()
                df.index = pd.to_datetime(df.index, format='%Y')
                df = df.dropna()
                df = df.drop('Indicator', axis = 1)
                df_original = df.copy()
                
                for lag in range(1, 4):  
                    df[f'lag_{lag}'] = df['Value'].shift(lag)
                
                df['expanding_mean'] = df['Value'].expanding().mean()
                df['expanding_std'] = df['Value'].expanding().std()
                df['expanding_max'] = df['Value'].expanding().max()
                df['expanding_min'] = df['Value'].expanding().min()
                
                window_size = 3  
                df['rolling_mean'] = df['Value'].rolling(window=window_size, min_periods=1).mean()
                df['rolling_std'] = df['Value'].rolling(window=window_size, min_periods=1).std()
                df['rolling_max'] = df['Value'].rolling(window=window_size, min_periods=1).max()
                df['rolling_min'] = df['Value'].rolling(window=window_size, min_periods=1).min()
                
                
                #df = df.dropna()
                train_size = int(len(df) * 0.8)
                
                train_xgb, test_xgb = df.iloc[:train_size], df.iloc[train_size:]
                feature_columns = ['rolling_mean','rolling_max', 'rolling_min','rolling_std' ,
                                   'expanding_mean', 'expanding_max', 'expanding_min','expanding_std',
                                   'lag_1', 'lag_2', 'lag_3', ]
                train_x, train_y = train_xgb[feature_columns], train_xgb['Value']
                test_x, test_y = test_xgb[feature_columns], test_xgb['Value']
                
                model_errors_rmse[(country, indicator)] = {}
                model_errors_rmse[(country, indicator)]['XGBoost'], xgb_pred = train_xgboost(train_x, train_y, test_x, test_y , country,indicator)
                
                
                sorted_models = sorted(model_errors_rmse[(country, indicator)].items(), key=lambda x: x[1])
                log_current_data = []
                for rank, (model_name, rmse) in enumerate(sorted_models, start=1):
                    log_data.append([country, indicator, model_name, rmse, rank])
                    log_current_data.append([country, indicator, model_name, rmse, rank])


from datetime import datetime
model ="XGBoost"
log_dir = f"../data/{model}_train"
os.makedirs(log_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d--%H-%M")
log_filename = os.path.join(log_dir, f"{model}_error_log_{timestamp}.csv")

log_df = pd.DataFrame(log_data, columns=['Country', 'Indicator', 'Model', 'RMSE', 'Rank'])
log_df.to_csv(log_filename, index=False)

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFE
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

def save_plot(train, test_y, predictions, country, indicator, model_name):
    """Function to save the plot in both Indicators and Countries folders."""

    model_colors = {
    "ARIMA": "blue",
    "Holt_Winters": "yellow",
    "LSTM": "black",
    "XGBoost": "pink",
    "Prophet": "brown"
}

    # Plotting predicted vs actual
    plt.figure(figsize=(10, 6))
    if model_name == "Prophet":
        plt.plot(train['ds'], train['y'], label='Train Data', color='green', linestyle='--')
        plt.plot(test_y['ds'], test_y['y'], label='Actual', color='red', linestyle='--')
        plt.plot(test_y['ds'], predictions, label=f'Predicted({model_name})', color=f'{model_colors["Prophet"]}', 
                 linestyle='-', marker='o')
    else:
        plt.plot(train.index, train, label='Train Data', color='green', linestyle='--')
        plt.plot(test_y.index, test_y, label='Actual', color='red', linestyle='--')
        plt.plot(test_y.index, predictions, label=f'Predicted({model_name})', color=f'{model_colors[model_name]}', 
                 linestyle='-', marker='o')
    

    
    plt.title(f'Predicted({model_name}) vs Actual for {country} - {indicator}')
    plt.xlabel('Year')
    plt.ylabel('Value')
    plt.legend()

    # Create subfolder for the indicator if it doesn't exist
    indicator_folder = os.path.join('../images', 'model_plot', 'Indicators', indicator)
    os.makedirs(indicator_folder, exist_ok=True)
    
    # Save the plot in the Indicators folder with dynamic model name
    plot_filename_indicator = os.path.join(indicator_folder, f'{model_name}_{country.replace(" ", "_")}_{indicator.replace(" ", "_")}.png')
    plt.savefig(plot_filename_indicator)

    # Create subfolder for the country if it doesn't exist
    country_folder = os.path.join('../images', 'model_plot', 'Countries', country)
    os.makedirs(country_folder, exist_ok=True)
    
    # Save the same plot in the Countries folder with dynamic model name
    plot_filename_country = os.path.join(country_folder, f'{model_name}_{country.replace(" ", "_")}_{indicator.replace(" ", "_")}.png')
    plt.savefig(plot_filename_country)

    plt.close()

def train_xgboost(train_x, train_y, test_x, test_y, country, indicator):
    params_dir = os.path.join("../best_params", indicator)
    params_file = os.path.join(params_dir, f"XGBoost_{country}.json")
    best_params = None

    # Check if the parameter file exists
    if os.path.exists(params_file):
        print(f"Loading parameters from {params_file}...")
        with open(params_file, "r") as f:
            best_params = json.load(f)
    else:
        print("No saved parameters found, performing grid search...")

        # Define parameter grid for GridSearchCV
        param_grid = {
            'n_estimators': [50, 100, 200, 500, 700, 900],
            'learning_rate': [0.01, 0.05, 0.1],
            'max_depth': [3, 5, 7],
            'min_child_weight': [1, 3],
            'gamma': [0, 0.1, 0.3],
            'subsample': [0.5,0.7,0.9,1],
            'colsample_bytree': [0.8, 1.0]
        }

        # Initialize the model
        model = xgb.XGBRegressor(
            objective='reg:squarederror', 
            random_state=42
        )

        # Set up the grid search with cross-validation
        grid_search = GridSearchCV(
            estimator=model,
            param_grid=param_grid,
            cv=3,
            scoring='neg_root_mean_squared_error',
            verbose=0,
            n_jobs=-1
        )

        # Fit grid search
        grid_search.fit(train_x, train_y)

        # Get the best parameters from grid search
        best_params = grid_search.best_params_

        # Save the best parameters to JSON if found
        if best_params:
            os.makedirs(params_dir, exist_ok=True)
            with open(params_file, "w") as f:
                json.dump(best_params, f, indent=4)

    # Train the best model using loaded or selected parameters
    if best_params:
        best_model = xgb.XGBRegressor(
            n_estimators=best_params['n_estimators'],
            learning_rate=best_params['learning_rate'],
            max_depth=best_params['max_depth'],
            min_child_weight=best_params['min_child_weight'],
            gamma=best_params['gamma'],
            subsample=best_params['subsample'],
            colsample_bytree=best_params['colsample_bytree'],
            objective='reg:squarederror',
            random_state=42
        )

        # Perform Recursive Feature Elimination (RFE)
        rfe = RFE(estimator=best_model, n_features_to_select=3)
        rfe.fit(train_x, train_y)

        # Get the selected features
        selected_features = train_x.columns[rfe.support_]


        # Train the model on the selected features
        best_model.fit(train_x[selected_features], train_y, verbose=0)

        # Make predictions
        #predictions = best_model.predict(test_x[selected_features])

        predictions_rfe = best_model.predict(test_x[selected_features])

        # Calculate RMSE for the RFE model
        rmse_rfe = np.sqrt(mean_squared_error(test_y, predictions_rfe))

        # Feature importance analysis
        feature_importances = best_model.feature_importances_
        important_features = selected_features[feature_importances > 0.1]

        if len(important_features) > 0:
            # Retrain the model using only features with importance > 0.1
            best_model.fit(train_x[important_features], train_y, verbose=0)
            predictions_important = best_model.predict(test_x[important_features])

            # Calculate RMSE for the model trained on important features
            rmse_important = np.sqrt(mean_squared_error(test_y, predictions_important))

        if rmse_important >rmse_rfe:
            predictions = predictions_important
        else:
            predictions = predictions_rfe
        # Save the plot
        save_plot(train_y, test_y, predictions, country, indicator, model_name="XGBoost")

        return np.sqrt(mean_squared_error(test_y, predictions)), predictions
    else:
        print("No suitable parameters found.")
        return None, None







with open("../countries.json", "r") as f:
    country_names = json.load(f)

with open("../indicators.json", "r") as f:
    indicators = json.load(f)

data_folder = "../data/base"
model_errors_rmse = {}
log_data = []
country_indicators_plots = {}
for country, country_code in country_names.items():
    for indicator, indicator_code in indicators.items():
        filename = f"{country.replace(' ', '_')}_{indicator.replace(' ', '_')}.parquet"
        filepath = os.path.join(data_folder, filename)
        
        if os.path.exists(filepath):
            df = pd.read_parquet(filepath)
            if 'Year' in df.columns and 'Value' in df.columns:
                df = df.set_index('Year').sort_index()
                df.index = pd.to_datetime(df.index, format='%Y')
                df = df.dropna()
                df = df.drop('Indicator', axis = 1)
                df_original = df.copy()
                
                for lag in range(1, 4):  
                    df[f'lag_{lag}'] = df['Value'].shift(lag)
                
                df['expanding_mean'] = df['Value'].expanding().mean()
                df['expanding_std'] = df['Value'].expanding().std()
                df['expanding_max'] = df['Value'].expanding().max()
                df['expanding_min'] = df['Value'].expanding().min()
                
                window_size = 3  
                df['rolling_mean'] = df['Value'].rolling(window=window_size, min_periods=1).mean()
                df['rolling_std'] = df['Value'].rolling(window=window_size, min_periods=1).std()
                df['rolling_max'] = df['Value'].rolling(window=window_size, min_periods=1).max()
                df['rolling_min'] = df['Value'].rolling(window=window_size, min_periods=1).min()
                
                
                #df = df.dropna()
                train_size = int(len(df) * 0.8)
                
                train_xgb, test_xgb = df.iloc[:train_size], df.iloc[train_size:]
                feature_columns = ['rolling_mean','rolling_max', 'rolling_min','rolling_std' ,
                                   'expanding_mean', 'expanding_max', 'expanding_min','expanding_std',
                                   'lag_1', 'lag_2', 'lag_3', ]
                train_x, train_y = train_xgb[feature_columns], train_xgb['Value']
                test_x, test_y = test_xgb[feature_columns], test_xgb['Value']
                
                model_errors_rmse[(country, indicator)] = {}
                model_errors_rmse[(country, indicator)]['XGBoost'], xgb_pred = train_xgboost(train_x, train_y, test_x, test_y , country,indicator)
                
                
                sorted_models = sorted(model_errors_rmse[(country, indicator)].items(), key=lambda x: x[1])
                log_current_data = []
                for rank, (model_name, rmse) in enumerate(sorted_models, start=1):
                    log_data.append([country, indicator, model_name, rmse, rank])
                    log_current_data.append([country, indicator, model_name, rmse, rank])


from datetime import datetime
model ="XGBoost"
log_dir = f"../data/{model}_train"
os.makedirs(log_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d--%H-%M")
log_filename = os.path.join(log_dir, f"{model}_error_log_{timestamp}.csv")

log_df = pd.DataFrame(log_data, columns=['Country', 'Indicator', 'Model', 'RMSE', 'Rank'])
log_df.to_csv(log_filename, index=False)
